### Check passes in `init` stage of PassManager

In [910]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.providers.fake_provider import FakeTokyo

# Get the passes from the init stage of a preset PassManager instance,
# which is the default pass managers used by the transpile() function.
backend = FakeTokyo()

pm = generate_preset_pass_manager(0) #backend = backend is optional
print("\n")
print("passes in init stage:", pm.init.passes())

# When the backend is not specified, pm.init.passes only contains only an analysis pass
# to detect if the DAG contains a specific instruction. Otherwise, it also contains \
# unitary synthesis, high level synthesis, and unroll_3q_or_more.



passes in init stage: [{'passes': [<qiskit.transpiler.passes.utils.contains_instruction.ContainsInstruction object at 0x000002A315A7A560>], 'flow_controllers': {}}]


### Example of PauliEvolutionGate instance

In [911]:
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.opflow import I, Z, X

# Build the evolution gate.
operator = (Z ^ Z) - 0.1 * (X ^ I)
evo = PauliEvolutionGate(operator, time=0.2)
print("Hamiltonian of PauliEvolutionGate object: ", evo.operator)


Hamiltonian of PauliEvolutionGate object:  SparsePauliOp(['ZZ', 'XI'],
              coeffs=[ 1. +0.j, -0.1+0.j])


### Example of Workflow for a 2-site Heisenberg model

The general form of a 1_D nearest neighbour Heisenberg model with open boundary conditions and free coefficients is 

$ \mathcal{H} = \sum_{i=1}^{n-1} \left( a_i X_i X_{i+1} + b_i Y_i Y_{i+1} + c_i Z_i Z_{i+1} \right)$ for some coefficients $\left\{ a_i, b_i, c_i \right\}_{i=1}^{n-1}$.

We will first try the example of a 3-site model with unit coefficients given its simplicity. 

In [912]:
from CQS.methods import Hamiltonian, Cartan, FindParameters
import numpy as np

# Define the system parameters.
sites = 3
model = 'heisenberg'
coefficient = 1

# Tuple will be used to generate the Hamiltonian.
modelTuple = [(coefficient, model)]

In [913]:
# Generate the Hamiltonian 1*(XX+YY+ZZ).
heisenbergH = Hamiltonian(sites, name=modelTuple)

# Print number of terms in Hamiltonian.
Hlen = len(heisenbergH.HCoefs)
print("number of Pauli strings in Hamiltonian:", Hlen)

# Print Hamiltonian.
heisenbergH.getHamiltonian(type='printText')

# For a custom Hamiltonian, can generate empty Hamiltonian object
# then add terms. See custom_hamiltonians.ipynb.

number of Pauli strings in Hamiltonian: 6
1 * XXI
1 * YYI
1 * ZZI
1 * IXX
1 * IYY
1 * IZZ


In [914]:
# Try to perform a Cartan involution on the Hamiltonian
# using the defauls evenOdd Decomposition.
# If H is not contained in the -1 eigenspace, raise an error.

try:
    heisenbergC = Cartan(heisenbergH)
except Exception as e:
    print(e)

In [915]:
print('g(H), the Hamiltonian Algebra: ', heisenbergC.g)
print('k, the +1 eigenspace of g under the involution: ', heisenbergC.k)
print('m, the -1 eigenspace of g under the involution: ', heisenbergC.m)
print('h: a Cartan subalgebra of m: ', heisenbergC.h)
# Note h is not unique in general; use seed to duplicate results.

# Check that g(H) is indeed the direct sum of k and m
# by checking the length of g(H) is the same as the sum of k and m.
assert(len(heisenbergC.g) == len(heisenbergC.k) + len(heisenbergC.m))

g(H), the Hamiltonian Algebra:  [(1, 3, 2), (1, 2, 3), (2, 3, 1), (2, 1, 3), (3, 2, 1), (3, 1, 2), (1, 1, 0), (2, 2, 0), (3, 3, 0), (0, 1, 1), (0, 2, 2), (0, 3, 3), (3, 0, 3), (2, 0, 2), (1, 0, 1)]
k, the +1 eigenspace of g under the involution:  [(1, 3, 2), (1, 2, 3), (2, 3, 1), (2, 1, 3), (3, 2, 1), (3, 1, 2)]
m, the -1 eigenspace of g under the involution:  [(1, 1, 0), (2, 2, 0), (3, 3, 0), (0, 1, 1), (0, 2, 2), (0, 3, 3), (3, 0, 3), (2, 0, 2), (1, 0, 1)]
h: a Cartan subalgebra of m:  [(1, 1, 0), (2, 2, 0), (3, 3, 0)]


In [916]:
# Generate the parameters via classical optimization of the cost function.
heisenbergP = FindParameters(heisenbergC)
heisenbergP.printResult()

Optimization terminated successfully.
         Current function value: -2.303234
         Iterations: 15
         Function evaluations: 18
         Gradient evaluations: 18
--- 0.058997154235839844 seconds ---
Optimization Error:
3.878611739499617e-12
Printing Results:
K elements 

1.1780972234347369  *XZY
-0.49346963349813827*XYZ
-0.4728223917547488 *YZX
-0.3077395585789813 *YXZ
1.105906361849959   *ZYX
-0.8674890304005023 *ZXY

 h elements: 
 
(-1.9999999999994478+0j) *XXI
(0.9999999999993744-0j)  *YYI
(-0.9999999999997908+0j) *ZZI
Normed Error |KHK - Exact|:
2.8694329212780605e-06


In [917]:
# Summarize above results.
print(heisenbergP.cartan.k)
print(heisenbergP.kCoefs)

print(heisenbergP.cartan.h)
print(heisenbergP.hCoefs)

# The normed error is the matrix norm between the unitary corresponding to the time-evolution
# operator of the cartan decomposed Hamiltonian and the exact Hamiltonian 
# for the special case of t=1.
# A generalization is described in next cell.

[(1, 3, 2), (1, 2, 3), (2, 3, 1), (2, 1, 3), (3, 2, 1), (3, 1, 2)]
[ 1.17809722 -0.49346963 -0.47282239 -0.30773956  1.10590636 -0.86748903]
[(1, 1, 0), (2, 2, 0), (3, 3, 0)]
[(-1.9999999999994478+0j), (0.9999999999993744-0j), (-0.9999999999997908+0j)]


The time-evolution propagator of the Hamiltonian corresponding to some time $t$ is 

$U(t) = e^{-i\mathcal{H}t} = K H K^\dag$, where:

- $K = \prod_{l} e^{i \times k_l \times kcoef_l}$ where $\{k_l\}$ are the basis elements of the +1 eigenspace and $\{kcoef_l\}$ are the coefficients for k found above,
- $H = \prod_{j} e^{-i \times h_j \times hcoef_j \times t}$ where $\{k_j\}$ are the basis elements of the Cartan subalgebra and $\{hcoef_j\}$ are the coefficients for h found above.

To verify this, we use qiskit to produce the explicit circuit for $KHK^\dag$ and then retrieve the overall unitary for some chosen $t$, `time_evolve`. We compare the resulting unitary with with the exact time evolution propagator (by measuring the matrix norm of their difference).

In [918]:
# Import paulilabel function from CQS.util.IO to convert a tuple of integers 
# representing a Pauli string into a string of letters, 
import CQS.util.IO as IO

# e.g (1, 3, 2) -> "X Z Y".
teststr = str(IO.paulilabel((1,3,2)))
teststr

'XZY'

In [919]:
from qiskit.circuit import QuantumCircuit
from qiskit.quantum_info import Pauli

time_evolve = 3

num_qubits = heisenbergH.sites
qc = QuantumCircuit(num_qubits)
qc.barrier()


# Note the order of gates:
# first add K^\dag
# then H
# then K.
# Also note the order of gates within each loop.


# Note order of qubits in qiskit (rightmost position is qubit 0) is the reverse of that in CQS.
# Also note PauliEvolutionGate implements exp(-1j...) by definition.

# K^\dag.
for ktuple, kcoef in list(zip(heisenbergC.k, heisenbergP.kCoefs)):
    kstring = str(IO.paulilabel(ktuple))
    gate = PauliEvolutionGate(Pauli(kstring[::-1]), time=kcoef) 
    qc.append(gate, range(num_qubits))

qc.barrier()

# H.
for htuple, hcoef in zip(heisenbergC.h, heisenbergP.hCoefs):
    hstring = str(IO.paulilabel(htuple))
    gate = PauliEvolutionGate(Pauli(hstring[::-1]), time=np.real(hcoef)*time_evolve) # WLOG convert complex to real
    qc.append(gate, range(num_qubits))

qc.barrier()

# K
for ktuple, kcoef in reversed(list(zip(heisenbergC.k, heisenbergP.kCoefs))):
    kstring = str(IO.paulilabel(ktuple))
    gate = PauliEvolutionGate(Pauli(kstring[::-1]), time=-kcoef)
    qc.append(gate, range(num_qubits))


qc.draw()

░ ┌───────────────────────┐┌─────────────────────────┐»
q_0: ─░─┤0                      ├┤0                        ├»
      ░ │                       ││                         │»
q_1: ─░─┤1 exp(-it YZX)(1.1781) ├┤1 exp(-it ZYX)(-0.49347) ├»
      ░ │                       ││                         │»
q_2: ─░─┤2                      ├┤2                        ├»
      ░ └───────────────────────┘└─────────────────────────┘»
«     ┌─────────────────────────┐┌─────────────────────────┐»
«q_0: ┤0                        ├┤0                        ├»
«     │                         ││                         │»
«q_1: ┤1 exp(-it XZY)(-0.47282) ├┤1 exp(-it ZXY)(-0.30774) ├»
«     │                         ││                         │»
«q_2: ┤2                        ├┤2                        ├»
«     └─────────────────────────┘└─────────────────────────┘»
«     ┌───────────────────────┐┌─────────────────────────┐ ░ »
«q_0: ┤0                      ├┤0                        ├─░─»
«     │                       ││                         │ ░ »
«q_1: ┤1 exp(-it XYZ)(1.1059) ├┤1 exp(-it YXZ)(-0.86749) ├─░─»
«     │                       ││                         │ ░ »
«q_2: ┤2                      ├┤2                        ├─░─»
«     └───────────────────────┘└─────────────────────────┘ ░ »
«     ┌───────────────────┐┌──────────────────┐┌───────────────────┐ ░ »
«q_0: ┤0                  ├┤0                 ├┤0                  ├─░─»
«     │                   ││                  ││                   │ ░ »
«q_1: ┤1 exp(-it IXX)(-6) ├┤1 exp(-it IYY)(3) ├┤1 exp(-it IZZ)(-3) ├─░─»
«     │                   ││                  ││                   │ ░ »
«q_2: ┤2                  ├┤2                 ├┤2                  ├─░─»
«     └───────────────────┘└──────────────────┘└───────────────────┘ ░ »
«     ┌────────────────────────┐┌────────────────────────┐»
«q_0: ┤0                       ├┤0                       ├»
«     │                        ││                        │»
«q_1: ┤1 exp(-it YXZ)(0.86749) ├┤1 exp(-it XYZ)(-1.1059) ├»
«     │                        ││                        │»
«q_2: ┤2                       ├┤2                       ├»
«     └────────────────────────┘└────────────────────────┘»
«     ┌────────────────────────┐┌────────────────────────┐»
«q_0: ┤0                       ├┤0                       ├»
«     │                        ││                        │»
«q_1: ┤1 exp(-it ZXY)(0.30774) ├┤1 exp(-it XZY)(0.47282) ├»
«     │                        ││                        │»
«q_2: ┤2                       ├┤2                       ├»
«     └────────────────────────┘└────────────────────────┘»
«     ┌────────────────────────┐┌────────────────────────┐
«q_0: ┤0                       ├┤0                       ├
«     │                        ││                        │
«q_1: ┤1 exp(-it ZYX)(0.49347) ├┤1 exp(-it YZX)(-1.1781) ├
«     │                        ││                        │
«q_2: ┤2                       ├┤2                       ├
«     └────────────────────────┘└────────────────────────┘

In [920]:
from qiskit import Aer, transpile
unitary_simulator = Aer.get_backend('unitary_simulator')

# Transpile circuit for simulator.
qc_transp = transpile(qc, unitary_simulator)

# Execute circuit on simulator.
final_unitary_cartan = unitary_simulator.run(qc_transp).result().get_unitary()

In [921]:
# compare with ideal result
import scipy
from qiskit.quantum_info import SparsePauliOp

H = SparsePauliOp(["XXI", "YYI", "ZZI", "IXX", "IYY", "IZZ"], 
                  np.array([1, 1, 1, 1, 1, 1])).to_matrix()
propagator = scipy.linalg.expm(-1j*H*time_evolve)

matrix_norm = np.linalg.norm(propagator - final_unitary_cartan)
print("matrix norm of the difference between the ideal propagator and that obtained by cartan decomposition: \n", matrix_norm)

matrix norm of the difference between the ideal propagator and that obtained by cartan decomposition: 
 7.769560961856694e-07


### Do the same thing but with statevector_simulator and some random initial state

In [922]:
from qiskit.circuit import QuantumCircuit
from qiskit.quantum_info import Pauli, random_statevector

random_seed = 10
time_evolve = 3

num_qubits = heisenbergH.sites
qc = QuantumCircuit(num_qubits)
init_statevec = random_statevector(2**num_qubits, seed = random_seed)
qc.initialize(init_statevec)
qc.barrier()


# Note the order of gates:
# first add K^\dag
# then H
# then K.
# Also note the order of gates within each loop.

# Note order of qubits in qiskit (rightmost position is qubit 0) is the reverse of that in CQS.
# Also note PauliEvolutionGate implements exp(-1j...) by definition.

# K^\dag.
for ktuple, kcoef in list(zip(heisenbergC.k, heisenbergP.kCoefs)):
    kstring = str(IO.paulilabel(ktuple))
    gate = PauliEvolutionGate(Pauli(kstring[::-1]), time=kcoef) 
    qc.append(gate, range(num_qubits))

qc.barrier()

# H.
for htuple, hcoef in zip(heisenbergC.h, heisenbergP.hCoefs):
    hstring = str(IO.paulilabel(htuple))
    gate = PauliEvolutionGate(Pauli(hstring[::-1]), time=np.real(hcoef)*time_evolve) # WLOG convert complex to real
    qc.append(gate, range(num_qubits))

qc.barrier()

# K.
for ktuple, kcoef in reversed(list(zip(heisenbergC.k, heisenbergP.kCoefs))):
    kstring = str(IO.paulilabel(ktuple))
    gate = PauliEvolutionGate(Pauli(kstring[::-1]), time=-kcoef)
    qc.append(gate, range(num_qubits))


qc.draw()

»
q_0: »
     »
q_1: »
     »
q_2: »
     »
«     ┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐»
«q_0: ┤0                                                                                                                                                          ├»
«     │                                                                                                                                                           │»
«q_1: ┤1 Initialize(-0.07199+0.036391j,0.4594-0.12746j,0.075204-0.14684j,-0.27603+0.44592j,-0.27622-0.14241j,0.011144-0.5371j,0.10809-0.20537j,0.14423-0.064103j) ├»
«     │                                                                                                                                                           │»
«q_2: ┤2                                                                                                                                                          ├»
«     └───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘»
«      ░ ┌───────────────────────┐┌─────────────────────────┐»
«q_0: ─░─┤0                      ├┤0                        ├»
«      ░ │                       ││                         │»
«q_1: ─░─┤1 exp(-it YZX)(1.1781) ├┤1 exp(-it ZYX)(-0.49347) ├»
«      ░ │                       ││                         │»
«q_2: ─░─┤2                      ├┤2                        ├»
«      ░ └───────────────────────┘└─────────────────────────┘»
«     ┌─────────────────────────┐┌─────────────────────────┐»
«q_0: ┤0                        ├┤0                        ├»
«     │                         ││                         │»
«q_1: ┤1 exp(-it XZY)(-0.47282) ├┤1 exp(-it ZXY)(-0.30774) ├»
«     │                         ││                         │»
«q_2: ┤2                        ├┤2                        ├»
«     └─────────────────────────┘└─────────────────────────┘»
«     ┌───────────────────────┐┌─────────────────────────┐ ░ »
«q_0: ┤0                      ├┤0                        ├─░─»
«     │                       ││                         │ ░ »
«q_1: ┤1 exp(-it XYZ)(1.1059) ├┤1 exp(-it YXZ)(-0.86749) ├─░─»
«     │                       ││                         │ ░ »
«q_2: ┤2                      ├┤2                        ├─░─»
«     └───────────────────────┘└─────────────────────────┘ ░ »
«     ┌───────────────────┐┌──────────────────┐┌───────────────────┐ ░ »
«q_0: ┤0                  ├┤0                 ├┤0                  ├─░─»
«     │                   ││                  ││                   │ ░ »
«q_1: ┤1 exp(-it IXX)(-6) ├┤1 exp(-it IYY)(3) ├┤1 exp(-it IZZ)(-3) ├─░─»
«     │                   ││                  ││                   │ ░ »
«q_2: ┤2                  ├┤2                 ├┤2                  ├─░─»
«     └───────────────────┘└──────────────────┘└───────────────────┘ ░ »
«     ┌────────────────────────┐┌────────────────────────┐»
«q_0: ┤0                       ├┤0                       ├»
«     │                        ││                        │»
«q_1: ┤1 exp(-it YXZ)(0.86749) ├┤1 exp(-it XYZ)(-1.1059) ├»
«     │                        ││                        │»
«q_2: ┤2                       ├┤2                       ├»
«     └────────────────────────┘└────────────────────────┘»
«     ┌────────────────────────┐┌────────────────────────┐»
«q_0: ┤0                       ├┤0                       ├»
«     │                        ││                        │»
«q_1: ┤1 exp(-it ZXY)(0.30774) ├┤1 exp(-it XZY)(0.47282) ├»
«     │                        ││                        │»
«q_2: ┤2                       ├┤2                       ├»
«     └────────────────────────┘└────────────────────────┘»
«     ┌────────────────────────┐┌────────────────────────┐
«q_0: ┤0                       ├┤0                       ├
«     │         

In [923]:
from qiskit import Aer, transpile

# Retrieve statevector simulator.
statevec_simulator = Aer.get_backend('statevector_simulator')

# Transpile circuit for statevector simulator.
qc_transp = transpile(qc, statevec_simulator)

# Execute circuit on statevector simulator.
final_statevec_cartan = statevec_simulator.run(qc_transp).result().get_statevector()

In [924]:
# Compare with ideal result.
import scipy
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info import state_fidelity

H = SparsePauliOp(["XXI", "YYI", "ZZI", "IXX", "IYY", "IZZ"], 
                  np.array([1, 1, 1, 1, 1, 1])).to_matrix()
propagator = scipy.linalg.expm(-1j*H*time_evolve)
final_statevec_ideal = propagator @ init_statevec.data

# Note: do not use np.inner as that does not perform complex conjugation!!!
fidelity = state_fidelity(final_statevec_ideal, final_statevec_cartan)
print("state fidelity between the ideal statevector and that obtained by evolving under a cartan decomposed circuit: \n", fidelity)

state fidelity between the ideal statevector and that obtained by evolving under a cartan decomposed circuit: 
 0.999999999999905
